In [1]:
import pandas as pd 
import urllib
from sqlalchemy import create_engine 
#import sqlalchemy as db
#import pyodbc 
from sqlalchemy.sql import text
from openpyxl import load_workbook
#import pytz 
from datetime import datetime, timedelta
from pathlib import Path
import warnings
warnings.filterwarnings("ignore", 'This pattern has match groups')
import os

In [2]:
def store_data(df):
    current_date = datetime.now().strftime('%Y-%m-%d')
    filename = (r"/home/Notebook_Scripts/ML_MODEL_2020/Output/Hans_"+current_date+"_data.xlsx")
    df = pd.read_excel(filename)
    df['Sub_category'] =df['error_category']
    pd.set_option('mode.chained_assignment', None)
    df2= df['Title'].str.lower()
    spec_chars = ['"',"'","[","]"]
    for char in spec_chars:
        df['Sub_category'] = df['Sub_category'].str.replace(char,'')
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    df['desc'] = df['Title'].replace("_",'', regex=True)
    df['desc'] = df['desc'].replace(to_replace= r'\\', value= '', regex=True)
    df['desc'] = df['desc'].replace(to_replace= r' ', value= '', regex=True)
    df['desc'] = df['desc'].str.lower()
    df['Sub_category'][(df['desc'].str.contains("logicaldiskspace"))] = 'Disk space utilization error'
    df['Sub_category'][(df['desc'].str.contains("Ignoringbrokenpanelcolor"))] = 'z-unmapped'
    df['Sub_category'][(df['desc'].str.contains("cannotconnecttodatabase"))] = 'Cannot connect to database'
    df['Sub_category'][(df['desc'].str.contains('services:""'))] = 'Services not running'
    df['Sub_category'][(df['desc'].str.contains("computerstatuscritical|computerstatusmajor"))]= 'z-unmapped'
    df['Sub_category'][(df['Title'].str.contains("SQL Server Scheduled"))]= 'App related Job Failure'
    df['Sub_category'][(df['desc'].str.contains("thedatabasestatusisunknown|thepluggabledatabase|thepluggabledatabase"))] = 'Database status unknown/down'
    df['Sub_category'][(df['Title'].str.contains("LISTENER|ThecurrentstatusofthetargetisDown"))] = 'Listener down/timeout'
    df['Sub_category'][(df['Title'].str.contains('Tablespace'))] = 'Tablespace threshold alert'
    df['Sub_category'][(df['desc'].str.contains('%ofarchivearea'))] = 'Archive mountpoint threshold'
    df['Sub_category'][(df['desc'].str.contains('abnormalvoltagerange'))] = 'Abnormal Voltage Range'
    df['Sub_category'][(df['desc'].str.contains('dbrecoveryfiledestsize'))] = 'DB recovery file destination threshold'
    df['Sub_category'][(df['desc'].str.contains('backupfailed'))] = 'Backup failure'
    df['Sub_category'][(df['Title'].str.contains('INT DB-BACKUP-FAILURE'))] = 'Backup failure'
    df['Sub_category'][(df['Title'].str.contains('INT DB-BACKUP-NOTRUN'))] = 'Backup not triggered'
    df['Sub_category'][(df['Title'].str.contains('Agent_Unreachable|Agent_is_Unreachable'))] = 'Agent unreachable'
    df['Sub_category'][(df['Title'].str.contains('Partner_Agent_could_not_connect_to_Host'))]='Agent not able to connect to host'
    df['Sub_category'][(df['Title'].str.contains('A_data_block_was_corrupted'))] = 'A_data_block_was_corrupted'
    df['Sub_category'][(df['Title'].str.contains('Agent_has_stopped_monitoring'))] = 'Agent_has_stopped_monitoring'
    df['Sub_category'][(df['Title'].str.contains('available_space,_fallen'))] = 'available space fallen below threshold'
    df['Sub_category'][(df['Title'].str.contains('Disk_Group_RECO'))] = 'Disk_Group_RECO'
    df['Sub_category'][(df['Title'].str.contains('Failed_to_connect_to_ASM_instance'))] = 'Failed_to_connect_to_ASM_instance'
    df['Sub_category'][(df['Title'].str.contains('Failed_to_connect_to_database'))] = 'Failed_to_connect_to_database'
    df['Sub_category'][(df['Title'].str.contains('Metrics_Process_Limit'))] = 'Metrics_Process_Limit'
    df['Sub_category'][(df['Title'].str.contains('redo_not_yet_available_on_this_standby_database'))] = 'redo_not_yet_available_on_standby_database'
    df['Sub_category'][(df['Title'].str.contains('standby_database_is_approximately'))] = 'Standby DB behind primary DB'
    df['Sub_category'][(df['Title'].str.contains('The_archiver_hung_at_time'))] = 'The_archiver_hung'
    df['Sub_category'][(df['Title'].str.contains('The_Data_Guard_status'))] = 'Data_Guard_status_error'
    df['Sub_category'][(df['Title'].str.contains('service entered failed state'))] = 'Service entered failed state'
    df['Sub_category'][(df['Title'].str.contains('Solve the recurrence root or create Problem Ticket'))] = 'Solve the recurrence root or create Problem Ticket'
    df['Sub_category'][(df['Title'].str.contains("unassigned disks"))] = 'unassigned disks'
    df['Sub_category'][(df['desc'].str.contains("aggregatecapacityissues|hawkeye|serviceprocessor|storagefailover|storagevirtualmachine"))] = 'NetApp issue'
    df['Sub_category'][(df['desc'].str.contains("cannotassignrequestedaddress|sd_journal_get_cursor()failed"))] = 'Sys log Error'
    df['Sub_category'][(df['Title'].str.contains("rman backup|kill rman"))] = 'RMAN Backup issue'
    df['Sub_category'][(df['desc'].str.contains("pingid"))] = 'Ping id unpair'
    df['Sub_category'][(df['desc'].str.contains('service:"'))] = 'Services not running'
    df['Sub_category'][(df['desc'].str.contains('service:"citrix'))] = 'Citrix service not running'
    df['Sub_category'][(df['desc'].str.contains("vmwaretool"))] = 'VMware Tools not running'
    df['Sub_category'][(df['desc'].str.contains("windowstime"))] = 'Windows Time not running'
    df['Sub_category'][(df['desc'].str.contains("passwordhasbeenchanged"))] = 'Access issue- password has been changed'
    df['Sub_category'][(df['desc'].str.contains("distributedtransactioncoordinator"))] = 'Distributed Transaction Coordinator'
    df['Sub_category'][(df['desc'].str.contains("crowdstrike"))] = 'Crowdstrike Issue'
    df['Sub_category'][(df['desc'].str.contains("winaudit"))] = 'WinAudit error'
    df['Sub_category'][(df['Title'].str.contains("syscall_wrappers"))] = 'z-unmapped'
    df['Sub_category'][(df['desc'].str.contains("volumeefficiencyoperationencounteredtheerror"))] = 'Volume efficiency operation encountered the error'
    df['Sub_category'][(df['desc'].str.contains("gnutls"))] = 'GnuTLS error'
    df['Sub_category'][(df['desc'].str.contains("recoveryinfocollector"))] = 'Recovery Info Collector'
    df['Sub_category'][(df['desc'].str.contains("databaseintegritycheck"))] = 'Database Integrity Check'
    df['Sub_category'][(df['desc'].str.contains("sqlsystemdbmaint"))] = 'System DB Maint'
    df['Sub_category'][(df['desc'].str.contains("addedtogroup"))] = 'Access issue-added to group'
    df['Sub_category'][(df['desc'].str.contains("indexoptimize"))] = 'Index Optimize'
    df['Sub_category'][(df['desc'].str.contains("updatestatistics-userdatabases"))] = 'Update Statistics - USER_DATABASES'
    df['Sub_category'][(df['desc'].str.contains("libraryissues|tapelibraryisdown|tapelibrary|tape"))] = 'Tape Library issue'
    df['Sub_category'][(df['desc'].str.contains("mediasagent|wentofflinestate|mediaagent|mounterror"))] = 'Device error'
    df['Sub_category'][(df['desc'].str.contains("sqlserverduringaread|sqlserverduringawriteatoffset "))] = 'DB FILE R/W Error'
    df['Sub_category'][(df['desc'].str.contains("diske:datasqlsupportss")) & (df['error_category'].str.contains('Disk space utilization error'))] = 'Backup Disk Drive Error'
    df['Sub_category'][(df['desc'].str.contains("diske:datasqlsd")) & (df['error_category'].str.contains('Disk space utilization error'))] = 'Data Disk Drive Error'
    df['Sub_category'][(df['desc'].str.contains("diske:datasqlst")) & (df['error_category'].str.contains('Disk space utilization error'))] = 'Temp DB Drive Error'
    df['Sub_category'][(df['desc'].str.contains("diske:datasqlsl")) & (df['error_category'].str.contains('Disk space utilization error'))] = 'DB Log File Drive Error'
    df['Sub_category'][(df['desc'].str.contains("databasebackup-userdatabases-diff")) & (df['error_category'].str.contains('SQL Server Scheduled job failed'))] = 'DatabaseBackup - USER_DATABASES – DIFF'
    df['Sub_category'][(df['desc'].str.contains("databasebackup-userdatabases-full")) & (df['error_category'].str.contains('SQL Server Scheduled job failed'))] = 'DatabaseBackup - USER_DATABASES - FULL'
    df['Sub_category'][(df['desc'].str.contains("databasebackup-userdatabases-log")) & (df['error_category'].str.contains('SQL Server Scheduled job failed'))] = 'DatabaseBackup - USER_DATABASES - LOG'
    df['Sub_category'][(df['desc'].str.contains("cannotconnecttodatabaseacm"))] = 'Cannot connect to database ACM'
    df['Sub_category'][(df['desc'].str.contains("cannotconnecttodatabasecitrix"))] = 'Cannot connect to database citrix'
    df['Sub_category'][(df['desc'].str.contains("cannotconnecttodatabaseidms"))] = 'Cannot connect to database IDMS'
    df['Sub_category'][(df['desc'].str.contains("cannotconnecttodatabasemsdb"))] = 'Cannot connect to database msdb'
    df['Sub_category'][(df['desc'].str.contains("cannotconnecttodatabasesap"))] = 'Cannot connect to database SAP'
    df['Sub_category'][(df['desc'].str.contains("cannotconnecttodatabasesqladmin"))] = 'Cannot connect to database SQL admin'
    df['Sub_category'][(df['desc'].str.contains("cannotconnecttodatabasesrbi"))] = 'Cannot connect to database SRBI'
    df['Sub_category'][(df['desc'].str.contains("cannotconnecttodatabasetempdb"))] = 'Cannot connect to database tempdb'
    df['Sub_category'][(df['desc'].str.contains('service:"windowstime"isnotrunning'))] = 'Windows Time not running'
    df = df[['IncidentID','OpenTime','Title','error_category','Sub_category']]
    df.to_excel(r"/home/Notebook_Scripts/ML_MODEL_2020/Output/Hans_"+current_date+"_data.xlsx")
    print("Sub Categories  has been updated")
#     os.remove("/home/Notebook_Scripts/ML_MODEL_2020/Input/Hans_"+current_date+"_data.xlsx")
    %run ./excel2dash.ipynb

In [3]:
if __name__ == '__main__':
    current_date = datetime.now().strftime('%Y-%m-%d')
    filename = (r"/home/Notebook_Scripts/ML_MODEL_2020/Output/Hans_"+current_date+"_data.xlsx")
    df = pd.read_excel(filename)
    df['Sub_category'] =df['error_category']
    pd.set_option('mode.chained_assignment', None)
    store_data(df)

Sub Categories  has been updated
